In [ ]:
문제 풀기: ~ 4:50

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df = pd.read_csv('./test/edu_enrollees.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             19158 non-null  float64
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  float64
 13  target                  19158 non-null  float64
 14  Xgrp                    19158 non-null

# 전처리

In [3]:
step1=df.drop(columns=['city','company_size', 'company_type'])
df.shape, step1.shape

((19158, 15), (19158, 12))

In [4]:
obj_list=step1.columns[step1.dtypes == 'object']
obj_list

Index(['gender', 'relevant_experience', 'enrolled_university',
       'education_level', 'major_discipline', 'experience', 'last_new_job',
       'Xgrp'],
      dtype='object')

In [6]:
# step1.dtypes

In [7]:
step2=step1.dropna(subset=obj_list)
len(step1), len(step2)

(19158, 12477)

In [8]:
step2.experience.value_counts()

experience
>20    2635
5       833
4       744
6       738
10      713
9       682
3       678
7       674
8       525
15      514
2       512
11      492
14      424
16      388
12      370
13      301
17      272
19      233
1       227
18      217
<1      189
20      116
Name: count, dtype: int64

In [9]:
step3=step2[~(step2.experience.isin(['>20','<1']))].reset_index(drop=True)
step3['experience'] = step3['experience'].astype('int')
step3.dtypes

enrollee_id               float64
city_development_index    float64
gender                     object
relevant_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                  int32
last_new_job               object
training_hours            float64
target                    float64
Xgrp                       object
dtype: object

In [10]:
step3.last_new_job.value_counts()

last_new_job
1        4633
2        1687
>4       1313
never     818
3         618
4         584
Name: count, dtype: int64

In [11]:
step4=step3[~(step3['last_new_job'].isin(['>4','never']))].reset_index()
step4['last_new_job'] = step4['last_new_job'].astype('int')
step4.dtypes

index                       int64
enrollee_id               float64
city_development_index    float64
gender                     object
relevant_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                  int32
last_new_job                int32
training_hours            float64
target                    float64
Xgrp                       object
dtype: object

In [12]:
len(step4)

7522

In [13]:
base=step4.copy()

# Q1

In [14]:
q1=pd.crosstab(index=base.relevant_experience, columns=base.target, normalize='index')
q1

target,0.0,1.0
relevant_experience,,
Has relevant experience,0.784089,0.215911
No relevant experience,0.617127,0.382873


In [15]:
A=q1.loc['No relevant experience', 1.0]
B=q1.loc['Has relevant experience', 1.0]
round(A/B,2)

1.77

# Q2

In [16]:
list2=['gender','relevant_experience', 'enrolled_university',
       'education_level','major_discipline']

In [17]:
# q2=pd.get_dummies(base, columns=list2, drop_first=False)  # 시험 버전
q2=pd.get_dummies(base, columns=list2, drop_first=False, dtype='int')   # 최신버전
q2.head(2)

,index,enrollee_id,city_development_index,experience,last_new_job,training_hours,target,Xgrp,gender_Female,gender_Male,...,enrolled_university_no_enrollment,education_level_Graduate,education_level_Masters,education_level_Phd,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other,major_discipline_STEM
0,2,27107.0,0.92,7,1,46.0,1.0,train,0,1,...,1,1,0,0,0,0,0,0,0,1
1,3,23853.0,0.92,5,1,108.0,0.0,train,0,1,...,1,1,0,0,0,0,0,0,0,1


In [34]:
base[list2].apply(lambda x: list(x.drop_duplicates().sort_values()))

gender                                             [Female, Male, Other]
relevant_experience    [Has relevant experience, No relevant experience]
enrolled_university    [Full time course, Part time course, no_enroll...
education_level                                 [Graduate, Masters, Phd]
major_discipline       [Arts, Business Degree, Humanities, No Major, ...
dtype: object

In [33]:
base[list2].apply(lambda x: list(x.drop_duplicates().sort_values())).explode()

gender                                  Female
gender                                    Male
gender                                   Other
relevant_experience    Has relevant experience
relevant_experience     No relevant experience
enrolled_university           Full time course
enrolled_university           Part time course
enrolled_university              no_enrollment
education_level                       Graduate
education_level                        Masters
education_level                            Phd
major_discipline                          Arts
major_discipline               Business Degree
major_discipline                    Humanities
major_discipline                      No Major
major_discipline                         Other
major_discipline                          STEM
dtype: object

In [20]:
drop_var=base[list2].apply(lambda x: list(x.drop_duplicates().sort_values())[-1]).reset_index(drop=False)
drop_var

,index,0
0,gender,Other
1,relevant_experience,No relevant experience
2,enrolled_university,no_enrollment
3,education_level,Phd
4,major_discipline,STEM


In [21]:
drop_list=drop_var['index'] + '_' + drop_var[0]
drop_list

0                                  gender_Other
1    relevant_experience_No relevant experience
2             enrolled_university_no_enrollment
3                           education_level_Phd
4                         major_discipline_STEM
dtype: object

In [22]:
q2_f=q2.drop(columns=drop_list)
q2.shape, q2_f.shape

((7522, 25), (7522, 20))

In [23]:
q2_f.columns

Index(['index', 'enrollee_id', 'city_development_index', 'experience',
       'last_new_job', 'training_hours', 'target', 'Xgrp', 'gender_Female',
       'gender_Male', 'relevant_experience_Has relevant experience',
       'enrolled_university_Full time course',
       'enrolled_university_Part time course', 'education_level_Graduate',
       'education_level_Masters', 'major_discipline_Arts',
       'major_discipline_Business Degree', 'major_discipline_Humanities',
       'major_discipline_No Major', 'major_discipline_Other'],
      dtype='object')

In [24]:
list2_2=q2_f.columns[2:]
job2=q2_f[list2_2].copy()

job2=job2.set_index(['target', 'Xgrp']).reset_index()
job2.head(2)

,target,Xgrp,city_development_index,experience,last_new_job,training_hours,gender_Female,gender_Male,relevant_experience_Has relevant experience,enrolled_university_Full time course,enrolled_university_Part time course,education_level_Graduate,education_level_Masters,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other
0,1.0,train,0.92,7,1,46.0,0,1,1,0,0,1,0,0,0,0,0,0
1,0.0,train,0.92,5,1,108.0,0,1,1,0,0,1,0,0,0,0,0,0


In [25]:
job2.shape

(7522, 18)

In [27]:
logit=LogisticRegression(C=100000, max_iter=1000, solver='liblinear', random_state=123)
logit.fit(job2.iloc[:,2:], job2.iloc[:,0])

LogisticRegression(C=100000, max_iter=1000, random_state=123,
                   solver='liblinear')

In [28]:
np.trunc(np.exp(logit.coef_).max()*100) / 100

1.67

# Q3

In [29]:
train=job2[job2.Xgrp == 'train']
test=job2[job2.Xgrp == 'test']

In [35]:
?KNeighborsClassifier

Init signature:
KNeighborsClassifier(
    n_neighbors=5,
    *,
    weights='uniform',
    algorithm='auto',
    leaf_size=30,
    p=2,
    metric='minkowski',
    metric_params=None,
    n_jobs=None,
)
Docstring:     
Classifier implementing the k-nearest neighbors vote.

Read more in the :ref:`User Guide <classification>`.

Parameters
----------
n_neighbors : int, default=5
    Number of neighbors to use by default for :meth:`kneighbors` queries.

weights : {'uniform', 'distance'}, callable or None, default='uniform'
    Weight function used in prediction.  Possible values:

    - 'uniform' : uniform weights.  All points in each neighborhood
      are weighted equally.
    - 'distance' : weight points by the inverse of their distance.
      in this case, closer neighbors of a query point will have a
      greater influence than neighbors which are further away.
    - [callable] : a user-defined function which accepts an
      array of distances, and returns an array of the same shape

In [30]:
knn=KNeighborsClassifier().fit(train.iloc[:, 2:], train.iloc[:, 0])
pred=knn.predict(test.iloc[:,2:])

In [31]:
from sklearn.metrics import accuracy_score

In [32]:
round(accuracy_score(test.target, pred),2)

0.71